In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
# from langchain_experimental.utilities import PythonREPL
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode

from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)

from dotenv import load_dotenv
load_dotenv()

# import nest_asyncio
# nest_asyncio.apply()

import os
from langchain.tools import Tool
import tiktoken
from datetime import datetime
from multiprocessing import Process, Queue
from difflib import unified_diff
from IPython.display import display, HTML

from llama_cloud_services import LlamaParse

from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
import qdrant_client
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from langchain_nomic.embeddings import NomicEmbeddings


from langchain_ollama import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = ChatOllama(
    model="llama 3.1",
    callback_manager=callback_manager,
    temperature=1
)

In [3]:
LLAMA_CLOUD_API_KEY="llx-nxrF5SQ5MnbMrKhrb1HqfdFA3YajxTDjtjfctsyebDSdBa7W"

# parser = LlamaParse(
#     api_key=LLAMA_CLOUD_API_KEY,
#     result_type="markdown"  # "markdown" and "text" are available
# )

# # use SimpleDirectoryReader to parse our file
# file_extractor = {".pdf": parser}
# documents = SimpleDirectoryReader(input_files=['/mnt/MainDrive/Codes/Heckeer/electronics-13-03233.pdf'], file_extractor=file_extractor).load_data()
# print(documents)

In [18]:
client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://:"
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key="",
)

In [19]:
# Settings.embed_model = NomicEmbeddings(model="nomic-embed-text-v1.5")
# vector_store = QdrantVectorStore(client=client, collection_name="Hecker")

# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents,
#     storage_context=storage_context,
#     # embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
# )


In [4]:
local_llm = "llama3.1"

In [ ]:
# #############SEARCH#####################
def get_search(query:str="", k:int=1): # get the top-k resources with google
    search = GoogleSearchAPIWrapper(k=k)
    def search_results(query):
        return search.results(query, k)
    tool = Tool(
        name="Google Search Snippets",
        description="Search Google for recent results.",
        func=search_results,
    )
    ref_text = tool.run(query)
    if 'Result' not in ref_text[0].keys():
        return ref_text
    else:
        return None

from langchain_community.document_transformers import Html2TextTransformer
from langchain_community.document_loaders import AsyncHtmlLoader
def get_page_content(link:str):
    loader = AsyncHtmlLoader([link])
    docs = loader.load()
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    if len(docs_transformed) > 0:
        return docs_transformed[0].page_content
    else:
        return None

def get_content(query):
    res = get_search(query, 1)
    if not res:
        print(">>> No good Google Search Result was found")
        return None
    search_results = res[0]
    link = search_results['link'] # title, snippet
    res = get_page_content(link)
    if not res:
        print(f">>> No content was found in {link}")
        return None
    retrieved_text = res
    trunked_texts = chunk_texts(retrieved_text, 1500)
    trunked_texts = [trunked_text.replace('\n', " ") for trunked_text in trunked_texts]
    return trunked_texts

In [21]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def chunk_text_by_sentence(text, chunk_size=2048):
    """Chunk the $text into sentences with less than 2k tokens."""
    sentences = text.split('. ')
    chunked_text = []
    curr_chunk = []
    for sentence in sentences:
        if num_tokens_from_string(". ".join(curr_chunk)) + num_tokens_from_string(sentence) + 2 <= chunk_size:
            curr_chunk.append(sentence)
        else:
            chunked_text.append(". ".join(curr_chunk))
            curr_chunk = [sentence]
    if curr_chunk:
        chunked_text.append(". ".join(curr_chunk))
    return chunked_text[0]

def chunk_text_front(text, chunk_size = 2048):
    '''
    get the first `trunk_size` token of text
    '''
    chunked_text = ""
    tokens = num_tokens_from_string(text)
    if tokens < chunk_size:
        return text
    else:
        ratio = float(chunk_size) / tokens
        char_num = int(len(text) * ratio)
        return text[:char_num]

def chunk_texts(text, chunk_size = 2048):
    '''
    trunk the text into n parts, return a list of text
    [text, text, text]
    '''
    tokens = num_tokens_from_string(text)
    if tokens < chunk_size:
        return [text]
    else:
        texts = []
        n = int(tokens/chunk_size) + 1
        
        part_length = len(text) // n
        
        extra = len(text) % n
        parts = []
        start = 0

        for i in range(n):

            end = start + part_length + (1 if i < extra else 0)
            parts.append(text[start:end])
            start = end
        return parts

In [5]:
from datetime import datetime
import os

def get_draft(question):
    # Getting the draft answer
    draft_prompt = '''
IMPORTANT:
Try to answer this question/instruction with step-by-step thoughts and make the answer more structural.
Use `\n\n` to split the answer into several paragraphs.
Just respond to the instruction directly. DO NOT add additional explanations or introducement in the answer unless you are asked to.
'''
    # openai_client = OpenAI(api_key=openai.api_key)
    draft_agent = ChatOllama(model=local_llm, callback_manager=callback_manager, format="json", temperature=1)
    response = draft_agent.invoke({
        "messages": [
            HumanMessage(content=f"{question}\n{draft_prompt}")
        ]
    })
    
    return response.content

In [6]:
from llama_cloud_services import LlamaParse
from llama_index.core import SimpleDirectoryReader
import tiktoken
from typing import List, Optional
from qdrant_client.models import Distance, VectorParams


class DocumentParser:
    def __init__(self, result_type="markdown",collection_name="Hecker"):
        self.parser = LlamaParse(result_type=result_type)
        self.file_extractor = {".pdf": self.parser}
        self.Settings.embed_model = NomicEmbeddings(model="nomic-embed-text-v1.5")
        self.client = qdrant_client.QdrantClient(host="localhost", port=6333)
        self.collection_name = collection_name
        
        try:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=384, distance=Distance.COSINE)
            )
        except:
            pass
        
    def parse_document(self, file_path):
        """Parse document using LlamaParse"""
        documents = SimpleDirectoryReader(
            input_files=[file_path], 
            file_extractor=self.file_extractor
        ).load_data()
        return [doc.text for doc in documents]
    
    def chunk_and_embed(self, texts, chunk_size=1500):
        """Chunk texts and create embeddings"""
        chunks = []
        for text in texts:
            # Split text into smaller chunks
            sentences = text.split('. ')
            current_chunk = []
            current_length = 0
            
            for sentence in sentences:
                if current_length + len(sentence) < chunk_size:
                    current_chunk.append(sentence)
                    current_length += len(sentence)
                else:
                    chunks.append('. '.join(current_chunk))
                    current_chunk = [sentence]
                    current_length = len(sentence)
            
            if current_chunk:
                chunks.append('. '.join(current_chunk))
        
        # Create embeddings
        embeddings = self.embeddings.embed_documents(chunks)
        return chunks, embeddings
    
    def index_document(self, file_path):
        """Process and index document"""
        # Parse document
        texts = self.parse_document(file_path)
        
        # Create chunks and embeddings
        chunks, embeddings = self.chunk_and_embed(texts)
        
        # Store in Qdrant
        self.client.upsert(
            collection_name=self.collection_name,
            points=[
                {"id": idx, "vector": emb, "payload": {"text": chunk}}
                for idx, (chunk, emb) in enumerate(zip(chunks, embeddings))
            ]
        )
        return chunks
    
    def semantic_search(self, query, top_k=2):
        """Search for relevant chunks"""
        query_vector = self.embeddings.embed_query(query)
        
        results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_vector,
            limit=top_k
        )
        
        return [hit.payload["text"] for hit in results]

In [7]:
def split_draft(draft, split_char = '\n\n'):
    paragraphs = draft.split(split_char)
    draft_paragraphs = [para for para in paragraphs if len(para)>5]
    # print(f"The draft answer has {len(draft_paragraphs)}")
    return draft_paragraphs

def split_draft_openai(question, answer, NUM_PARAGRAPHS = 4):
    split_prompt = f'''
Split the answer of the question into multiple paragraphs with each paragraph containing a complete thought.
The answer should be splited into less than {NUM_PARAGRAPHS} paragraphs.
Use ## as splitting char to seperate the paragraphs.
So you should output the answer with ## to split the paragraphs.
**IMPORTANT**
Just output the query directly. DO NOT add additional explanations or introducement in the answer unless you are asked to.
'''
    Ollama_client = ChatOllama(model=local_llm, callback_manager=callback_manager, format="json", temperature=1)
    splited_answer = Ollama_client.invoke({
        "messages": [
            HumanMessage(content=f"##Question: {question}\n\n##Response: {answer}\n\n##Instruction: {split_prompt}")
        ]
    })
    split_draft_paragraphs = split_draft(splited_answer, split_char = '##')
    return split_draft_paragraphs.content

In [8]:
def get_query(question, answer):
    query_prompt = '''
I want to verify the content correctness of the given question, especially the last sentences.
Please summarize the content with the corresponding question.
This summarization will be used as a query to search with Bing search engine.
The query should be short but need to be specific to promise Bing can find related knowledge or pages.
You can also use search syntax to make the query short and clear enough for the search engine to find relevant language data.
Try to make the query as relevant as possible to the last few sentences in the content.
**IMPORTANT**
Just output the query directly. DO NOT add additional explanations or introducement in the answer unless you are asked to.
'''
    # openai_client = OpenAI(api_key = openai.api_key)
    Ollama_client = ChatOllama(model=local_llm, callback_manager=callback_manager, format="json", temperature=1)
    query = Ollama_client.invoke({
        "messages": [
            HumanMessage(content=f"##Question: {question}\n\n##Response: {answer}\n\n##Instruction: {query_prompt}")
        ]
    })
    return query.content

In [9]:
def get_revise_answer(question, answer, content):
    revise_prompt = '''
I want to revise the answer according to retrieved related text of the question in WIKI pages.
You need to check whether the answer is correct.
If you find some errors in the answer, revise the answer to make it better.
If you find some necessary details are ignored, add it to make the answer more plausible according to the related text.
If you find the answer is right and do not need to add more details, just output the original answer directly.
**IMPORTANT**
Try to keep the structure (multiple paragraphs with its subtitles) in the revised answer and make it more structual for understanding.
Add more details from retrieved text to the answer.
Split the paragraphs with \n\n characters.
Just output the revised answer directly. DO NOT add additional explanations or annoucement in the revised answer unless you are asked to.
'''
    # openai_client = OpenAI(api_key = openai.api_key)
    Ollama_client = ChatOllama(model=local_llm, callback_manager=callback_manager, format="json", temperature=1)
    revised_answer = Ollama_client.invoke({
        "messages": [
            HumanMessage(content=f"##Existing Text in Wiki Web: {content}\n\n##Question: {question}\n\n##Answer: {answer}\n\n##Instruction: {revise_prompt}")
        ]
    })
    return revised_answer.content

def get_reflect_answer(question, answer):
    reflect_prompt = '''
Give a title for the answer of the question.
And add a subtitle to each paragraph in the answer and output the final answer using markdown format. 
This will make the answer to this question look more structured for better understanding.
**IMPORTANT**
Try to keep the structure (multiple paragraphs with its subtitles) in the response and make it more structual for understanding.
Split the paragraphs with \n\n characters.
Just output the revised answer directly. DO NOT add additional explanations or annoucement in the revised answer unless you are asked to.
'''

    Ollama_client = ChatOllama(model=local_llm, callback_manager=callback_manager, format="json", temperature=1)
    reflected_answer = Ollama_client.invoke({
        "messages": [
            HumanMessage(content=f"##Question:\n{question}\n\n##Answer:\n{answer}\n\n##Instruction:\n{reflect_prompt}")
        ]
    })
    return reflected_answer.content


In [10]:
def get_query_wrapper(q, question, answer):
    result = get_query(question, answer)
    q.put(result)

# def get_content_wrapper(q, query):
#     result = get_content(query)
#     q.put(result)

def get_revise_answer_wrapper(q, question, answer, content):
    result = get_revise_answer(question, answer, content)
    q.put(result)

def get_reflect_answer_wrapper(q, question, answer):
    result = get_reflect_answer(question, answer)
    q.put(result)

from multiprocessing import Process, Queue
def run_with_timeout(func, timeout, *args, **kwargs):
    q = Queue()  
    p = Process(target=func, args=(q, *args), kwargs=kwargs)
    p.start()
    p.join(timeout)
    if p.is_alive():
        print(f"{datetime.now()} [INFO] Function {str(func)} running timeout ({timeout}s), terminating...")
        p.terminate()
        p.join() 
        result = None
    else:
        print(f"{datetime.now()} [INFO] Function {str(func)} executed successfully.")
        result = q.get()  # 从队列中获取结果
    return result

In [11]:
from difflib import unified_diff
from IPython.display import display, HTML
import gradio as gr


def generate_diff_html(text1, text2):
    diff = unified_diff(text1.splitlines(keepends=True),
                        text2.splitlines(keepends=True),
                        fromfile='text1', tofile='text2')

    diff_html = ""
    for line in diff:
        if line.startswith('+'):
            diff_html += f"<div style='color:green;'>{line.rstrip()}</div>"
        elif line.startswith('-'):
            diff_html += f"<div style='color:red;'>{line.rstrip()}</div>"
        elif line.startswith('@'):
            diff_html += f"<div style='color:blue;'>{line.rstrip()}</div>"
        else:
            diff_html += f"{line.rstrip()}<br>"
    return diff_html

newline_char = '\n'

def rat(question):
    print(f"{datetime.now()} [INFO] Generating draft...")
    draft = get_draft(question)
    print(f"{datetime.now()} [INFO] Return draft.")
    # print(f"##################### DRAFT #######################")
    # print(draft)
    # print(f"#####################  END  #######################")

    print(f"{datetime.now()} [INFO] Processing draft ...")
    # draft_paragraphs = split_draft(draft)
    draft_paragraphs = split_draft_openai(question, draft)
    print(f"{datetime.now()} [INFO] Draft is splitted into {len(draft_paragraphs)} sections.")
    answer = ""
    for i, p in enumerate(draft_paragraphs):
        # print(str(i)*80)
        print(f"{datetime.now()} [INFO] Revising {i+1}/{len(draft_paragraphs)} sections ...")
        answer = answer + '\n\n' + p
        # print(f"[{i}/{len(draft_paragraphs)}] Original Answer:\n{answer.replace(newline_char, ' ')}")

        # query = get_query(question, answer)
        print(f"{datetime.now()} [INFO] Generating query ...")
        res = run_with_timeout(get_query_wrapper, 30, question, answer)
        if not res:
            print(f"{datetime.now()} [INFO] Generating query timeout, skipping...")
            continue
        else:
            query = res
        print(f">>> {i}/{len(draft_paragraphs)} Query: {query.replace(newline_char, ' ')}")

        print(f"{datetime.now()} [INFO] Crawling network pages ...")
        # content = get_content(query)
        res = run_with_timeout(get_content_wrapper, 30, query)
        if not res:
            print(f"{datetime.now()} [INFO] Parsing network pages timeout, skipping ...")
            continue
        else:
            content = res

        LIMIT = 2
        for j, c in enumerate(content):
            if  j >= LIMIT: # limit rge number of network pages
                break
            print(f"{datetime.now()} [INFO] Revising answers with retrieved network pages...[{j}/{min(len(content),LIMIT)}]")
            # answer = get_revise_answer(question, answer, c)
            res = run_with_timeout(get_revise_answer_wrapper, 30, question, answer, c)
            if not res:
                print(f"{datetime.now()} [INFO] Revising answers timeout, skipping ...")
                continue
            else:
                diff_html = generate_diff_html(answer, res)
                display(HTML(diff_html))
                answer = res
            print(f"{datetime.now()} [INFO] Answer revised [{j}/{min(len(content),3)}]")
        # print(f"[{i}/{len(draft_paragraphs)}] REVISED ANSWER:\n {answer.replace(newline_char, ' ')}")
        # print()
    res = run_with_timeout(get_reflect_answer_wrapper, 30, question, answer)
    if not res:
        print(f"{datetime.now()} [INFO] Reflecting answers timeout, skipping next steps...")
    else:
        answer = res
    return draft, answer


/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
page_title = "RAT: Retrieval Augmented Thoughts Elicit Context-Aware Reasoning in Long-Horizon Generation"
page_md = """
# RAT: Retrieval Augmented Thoughts Elicit Context-Aware Reasoning in Long-Horizon Generation

We explore how iterative revising a chain of thoughts with the help of information retrieval significantly improves large language models' reasoning and generation ability in long-horizon generation tasks, while hugely mitigating hallucination. In particular, the proposed method — retrieval-augmented thoughts (RAT) — revises each thought step one by one with retrieved information relevant to the task query, the current and the past thought steps, after the initial zero-shot CoT is generated.

Applying RAT to various base models substantially improves their performances on various long-horizon generation tasks; on average of relatively increasing rating scores by 13.63% on code generation, 16.96% on mathematical reasoning, 19.2% on creative writing, and 42.78% on embodied task planning.

Feel free to try our demo!

"""

def clear_func():
    return "", "", ""


with gr.Blocks(title = page_title) as demo:
   
    gr.Markdown(page_md)
    
    with gr.Row():
        file_input = gr.File(label="Upload Document")

    with gr.Row():
        chatgpt_box = gr.Textbox(
            label = "ChatGPT",
            placeholder = "Response from ChatGPT with zero-shot chain-of-thought.",
            elem_id = "chatgpt"
        )

    with gr.Row():
        stream_box = gr.Textbox(
            label = "Streaming",
            placeholder = "Interactive response with RAT...",
            elem_id = "stream",
            lines = 10,
            visible = False
        )
    
    with gr.Row():
        rat_box = gr.Textbox(
            label = "RAT",
            placeholder = "Final response with RAT ...",
            elem_id = "rat",
            lines = 6
        )

    with gr.Column(elem_id="instruction_row"):
        with gr.Row():
            instruction_box = gr.Textbox(
                label = "instruction",
                placeholder = "Enter your instruction here",
                lines = 2,
                elem_id="instruction",
                interactive=True,
                visible=True
            )
        # with gr.Row():
        #     model_radio = gr.Radio(["gpt-3.5-turbo", "gpt-4", "GPT-4-turbo"], elem_id="model_radio", value="gpt-3.5-turbo", 
        #                         label='GPT model', 
        #                         show_label=True,
        #                         interactive=True, 
        #                         visible=True) 
        #     openai_api_key_textbox = gr.Textbox(
        #         label='OpenAI API key',
        #         placeholder="Paste your OpenAI API key (sk-...) and hit Enter", 
        #         show_label=True, 
        #         lines=1, 
        #         type='password')
            
    # openai_api_key_textbox.change(set_openai_api_key,
    #     inputs=[openai_api_key_textbox],
    #     outputs=[])

    with gr.Row():
        submit_btn = gr.Button(
            value="submit", visible=True, interactive=True
        )
        clear_btn = gr.Button(
            value="clear", visible=True, interactive=True
        )
        regenerate_btn = gr.Button(
            value="regenerate", visible=True, interactive=True
        )

    submit_btn.click(
        fn = rat,
        inputs = [instruction_box, file_input],
        outputs = [chatgpt_box, rat_box]
    )

    clear_btn.click(
        fn = clear_func,
        inputs = [],
        outputs = [instruction_box, chatgpt_box, rat_box]
    )

    regenerate_btn.click(
        fn = rat,
        inputs = [instruction_box],
        outputs = [chatgpt_box, rat_box]
    )

    examples = gr.Examples(
        examples=[
            # "I went to the supermarket yesterday.", 
            # "Helen is a good swimmer."
            "Write a survey of retrieval-augmented generation in Large Language Models.",
            "Introduce Jin-Yong's life and his works.",
            "Summarize the American Civil War according to the timeline.",
            "Describe the life and achievements of Marie Curie"
            ],
        inputs=[instruction_box]
        )

demo.launch(server_name="0.0.0.0", debug=True)

/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/gradio/utils.py:1017: UserWarning: Expected 1 arguments for function <function rat at 0x77268f16cae0>, received 2.
  warnings.warn(
/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/gradio/utils.py:1025: UserWarning: Expected maximum 1 arguments for function <function rat at 0x77268f16cae0>, received 2.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/gradio/blocks.py", line 2098, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/gradio/blocks.py", line 1645, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/anyio/t

Keyboard interruption in main thread... closing server.
